In [ ]:
import numpy as np
import pandas as pd

In [ ]:
items = pd.read_csv("/content/items.csv")
interactions = pd.read_csv("/content/interactions.csv")

In [ ]:
interactions.sort_values('last_watch_dt', ascending=False)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
706895,603084,5543,2021-08-22,827,14.0
3308025,223918,3940,2021-08-22,16,0.0
3909230,436553,7622,2021-08-22,1373,50.0
3597440,849142,7793,2021-08-22,6112,100.0
1354688,957313,13018,2021-08-22,6223,91.0
...,...,...,...,...,...
2849666,296281,14612,2021-03-13,43,0.0
5192181,387623,8481,2021-03-13,6118,93.0
5283067,16660,15997,2021-03-13,2920,52.0
4120277,590611,9070,2021-03-13,513,8.0


In [ ]:
len(interactions['item_id'].unique())

15706

In [ ]:
len(interactions['user_id'].unique())

962179

In [ ]:
interactions[interactions['last_watch_dt'] > '2021-08-22']

,user_id,item_id,last_watch_dt,total_dur,watched_pct


In [ ]:
interactions[interactions['item_id'] == 5]

,user_id,item_id,last_watch_dt,total_dur,watched_pct
83532,912409,5,2021-05-30,1821,17.0
1141052,546847,5,2021-05-26,13624,50.0
1162628,875526,5,2021-08-11,816,0.0
1381614,1049472,5,2021-06-12,163,0.0
4648664,606194,5,2021-05-03,1241,17.0
5359521,272272,5,2021-08-10,3543,0.0


In [ ]:
top_items = interactions.groupby('item_id')['user_id'].count().reset_index()
top_items.columns=['item_id', 'views']
top_20_items = top_items.sort_values('views', ascending=False).head(20)

In [ ]:
top_items

,item_id,views
0,0,37
1,1,23
2,2,38
3,3,18
4,4,6
...,...,...
15701,16514,2
15702,16515,1
15703,16516,74
15704,16517,1


In [ ]:
top_20_items

,item_id,views
9906,10440,202457
14534,15297,193123
9242,9728,132865
13171,13865,122119
3941,4151,91167
3545,3734,74803
2525,2657,68581
4645,4880,55043
139,142,45367
6472,6809,40372


In [ ]:
top_20_items.to_csv("/content/top_20_items.csv")

In [ ]:
top_items.to_csv("/content/top_items.csv")

In [ ]:
top_items["item_id"].sample(10).to_list()

[13865, 4740, 15297, 12995, 9996, 14431, 1844, 3734, 4880, 2657]

#Weighted random reco

In [ ]:
top_weight_items = interactions.groupby('item_id')['total_dur'].sum().reset_index()
top_weight_items.columns=['item_id', 'total_dur']
top_weight_items# = top_weight_items.sort_values('views', ascending=False).head(20)

,item_id,total_dur
0,0,147169
1,1,163706
2,2,12655
3,3,87924
4,4,17419
...,...,...
15701,16514,9729
15702,16515,5151
15703,16516,89622
15704,16517,6599


In [ ]:
top_weight_items['weight'] = top_weight_items['total_dur'].apply(lambda x: np.log(x + 1))

In [ ]:
top_weight_items.sort_values('weight', ascending=False).head(20)

,item_id,total_dur,weight
9906,10440,5534082820,22.434192
14534,15297,4432354370,22.212197
3941,4151,959195844,20.681606
5874,6192,936851937,20.658036
14,14,865261665,20.578543
13171,13865,818712402,20.523243
9242,9728,781720945,20.477008
9494,9996,629719520,20.260785
15422,16228,579679341,20.177986
478,496,445214627,19.914067


In [ ]:
top_weight_items.to_csv("/content/top_weight_items.csv")

Попробуем удалять из рекомендаций те фильмы, которые уже были просмотрены:

In [ ]:
viewed_films = interactions.groupby('user_id')['item_id'].unique().reset_index()

In [ ]:
viewed_films.columns=['user_id', 'items_list_id']
viewed_films.head(10)

,user_id,items_list_id
0,0,"[7102, 14359, 15297, 6006, 9728, 12192]"
1,1,"[3669, 10440]"
2,2,"[7571, 3541, 15266, 13867, 12841, 10770, 4475,..."
3,3,"[12192, 9728, 16406, 15719, 10440, 3475, 2025,..."
4,4,"[4700, 6317]"
5,5,"[14397, 6445, 11437, 5651, 6167, 12466, 632, 8..."
6,7,[8710]
7,8,[6809]
8,9,"[9728, 10440, 142, 4151, 341, 15297, 2657, 999..."
9,10,"[10464, 13865, 1660]"


In [ ]:
viewed_films.to_csv("/content/viewed_films.csv")

In [ ]:
top_items_list = top_items.sort_values('views', ascending=False)["item_id"].to_list()

In [ ]:
top_items.sort_values('views', ascending=False)

,item_id,views
9906,10440,202457
14534,15297,193123
9242,9728,132865
13171,13865,122119
3941,4151,91167
...,...,...
8638,9098,1
2997,3159,1
12551,13219,1
2992,3154,1


In [ ]:
def top_popular_without_viewed(user_id, k=10):
    top = top_items_list
    if len(viewed_films[viewed_films["user_id"] == user_id]["items_list_id"].to_list()) > 0:
      viewed_films_user = viewed_films[viewed_films["user_id"] == 33]["items_list_id"].to_list()[0]
    else: viewed_films_user = []

    for i in range(len(viewed_films_user)):
      if viewed_films_user[i] in top:
        top.remove(viewed_films_user[i])

    reco = top_items_list[:k]
    return reco

In [ ]:
viewed_films['reco'] = viewed_films['user_id'].apply(lambda x: top_popular_without_viewed(x))

<ipython-input-220-8fb98a843fb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viewed_films_10['reco'] = viewed_films_10['user_id'].apply(lambda x: top_popular_without_viewed(x))
